# **Modeling**

#### **Import Packages**

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense
#from keras.optimizers import Adam
from keras import backend as K
from keras.utils import to_categorical

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


In [4]:
os.chdir(r'C:\Users\jdrel\OneDrive\Documents\Data_Science\Springboard\Capstone-2')

#### **Load the Data**

In [67]:
data = pd.read_csv('./data/interim/Small_df.csv')

clusters = pd.read_csv('./data/interim/Cluster_features.csv')['K=8']

dc = pd.concat([data, clusters], axis = 1)

In [24]:
X = dc.drop(['Unnamed: 0', 'labels'], axis = 1)
y = np.where(dc['labels'] == 'normal.', 0, 1)

In [82]:
XnoC = data.drop(['labels', 'Unnamed: 0'], axis = 1)
xnoctrain,xnoctest,ytrain,ytest = train_test_split(XnoC, y, test_size= 0.25, random_state = 35)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state = 42)

### **Neural Network**

In [11]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True #type: ignore
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)

In [30]:
if tf.test.gpu_device_name():
    print(f'Default GPU Device: {tf.test.gpu_device_name()}')
else:
    print("No GPU found. Using CPU instead.")

# Print backend engine and available devices
print('Backend engine:', K.backend())


No GPU found. Using CPU instead.
Backend engine: tensorflow


In [74]:
nnetnoc = Sequential()

nnetnoc.add(Dense(8, activation = 'relu', input_dim = XnoC.shape[1]))
nnetnoc.add(Dense(5, activation = 'relu'))
nnetnoc.add(Dense(3, activation = 'relu'))
nnetnoc.add(Dense(1, activation = 'sigmoid'))

nnetnoc.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

nnetnoc.fit(xnoctrain.astype('float32'), ytrain.astype('float32'), epochs = 3, batch_size = 50)

Epoch 1/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0360 - binary_accuracy: 0.9914
Epoch 2/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0132 - binary_accuracy: 0.9954
Epoch 3/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0124 - binary_accuracy: 0.9960


In [75]:
predsnoc = nnetnoc.predict(xnoctest)

predsnoc = np.where(predsnoc < .5, 0, 1)
print(classification_report(ytest, predsnoc))

3860/3860 [==============================] - 3s 890us/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     24387
           1       1.00      1.00      1.00     99118

    accuracy                           1.00    123505
   macro avg       1.00      0.99      0.99    123505
weighted avg       1.00      1.00      1.00    123505



In [83]:
nnet = Sequential()

nnet.add(Dense(8, activation = 'relu', input_dim = X.shape[1]))
nnet.add(Dense(5, activation = 'relu'))
nnet.add(Dense(3, activation = 'relu'))
nnet.add(Dense(1, activation = 'sigmoid'))

nnet.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

nnet.fit(X_train.astype('float32'), y_train.astype('float32'), epochs = 3, batch_size = 50)

preds = nnet.predict(X_test)

preds = np.where(preds < .5, 0, 1)
print(classification_report(y_test, preds))

Epoch 1/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.1439 - binary_accuracy: 0.9886
Epoch 2/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0174 - binary_accuracy: 0.9957
Epoch 3/3
3860/3860 [==============================] - 4s 902us/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     24285
           1       1.00      1.00      1.00     99220

    accuracy                           1.00    123505
   macro avg       1.00      0.99      0.99    123505
weighted avg       1.00      1.00      1.00    123505



In [79]:
preds = nnet.predict(X_test)

3860/3860 [==============================] - 3s 862us/step


In [80]:
preds = np.where(preds < .5, 0, 1)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     24285
           1       1.00      1.00      1.00     99220

    accuracy                           1.00    123505
   macro avg       0.99      0.99      0.99    123505
weighted avg       1.00      1.00      1.00    123505



In [49]:
data['labels'].value_counts(normalize=True)

smurf.              0.568378
neptune.            0.216997
normal.             0.196909
back.               0.004459
satan.              0.003216
ipsweep.            0.002524
portsweep.          0.002105
warezclient.        0.002065
teardrop.           0.001982
pod.                0.000534
nmap.               0.000468
guess_passwd.       0.000107
buffer_overflow.    0.000061
land.               0.000043
warezmaster.        0.000040
imap.               0.000024
rootkit.            0.000020
loadmodule.         0.000018
ftp_write.          0.000016
multihop.           0.000014
phf.                0.000008
perl.               0.000006
spy.                0.000004
Name: labels, dtype: float64

In [85]:
len(data['labels'].unique())

23

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, data['labels'], test_size=.25, random_state = 42)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train, num_classes = 23)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [98]:
nnet = Sequential()

nnet.add(Dense(38, activation = 'relu', input_dim = X.shape[1]))
nnet.add(Dense(31, activation = 'relu'))
nnet.add(Dense(23, activation = 'softmax'))

nnet.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

nnet.fit(X_train, y_train, epochs = 3, batch_size = 50)



Epoch 1/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0512 - accuracy: 0.9900
Epoch 2/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0147 - accuracy: 0.9963
Epoch 3/3
7411/7411 [==============================] - 9s 1ms/step - loss: 0.0133 - accuracy: 0.9967


In [106]:
#preds = nnet.predict(X_test)
pred = preds.argmax(axis = 1)
ints = label_encoder.classes_
predictions = [ints[i] for i in pred]
print(classification_report(y_test, predictions))

c:\Users\jdrel\miniconda3\envs\Capstone-2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jdrel\miniconda3\envs\Capstone-2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

           back.       0.98      0.99      0.99       540
buffer_overflow.       0.00      0.00      0.00        11
      ftp_write.       0.00      0.00      0.00         2
   guess_passwd.       0.88      0.78      0.82         9
           imap.       0.00      0.00      0.00         3
        ipsweep.       0.85      0.92      0.88       325
           land.       0.80      0.80      0.80         5
     loadmodule.       0.00      0.00      0.00         1
       multihop.       0.00      0.00      0.00         2
        neptune.       1.00      1.00      1.00     26666
           nmap.       0.50      0.02      0.04        50
         normal.       0.99      1.00      0.99     24285
           perl.       0.00      0.00      0.00         1
            phf.       0.00      0.00      0.00         2
            pod.       1.00      0.96      0.98        50
      portsweep.       0.88      0.97      0.92       278
        rootk

c:\Users\jdrel\miniconda3\envs\Capstone-2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### **Small Data**

#### **Big Data**

### **Random Forrest**

In [ ]:
# Create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=0)

# Fit the model to the data
rf.fit(X, y)